In [1]:
import pandas as pd
import numpy as np
import os, random, math, glob
from IPython.display import Image as IM
from IPython.display import clear_output
from matplotlib import pyplot as plt
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv('Train.csv')
train_df.head()

,ward,total_households,total_individuals,target,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,...,pw_03,pw_04,pw_05,pw_06,pw_07,pw_08,ADM4_PCODE,lat,lon,NL
0,41601001: Ward 1,1674.45058,5888.20750,16.773757,0.933841,0.000846,0.005490,0.000676,0.000000,0.001372,...,0.002848,0.007537,0.000000,0.012928,0,0,ZA4161001,-29.682270,24.734743,0.292039
1,41601002: Ward 2,1736.99230,6735.33812,21.496661,0.696940,0.001253,0.004402,0.000000,0.002301,0.001323,...,0.014566,0.057127,0.019092,0.004131,0,0,ZA4161002,-29.119311,24.757737,3.207775
2,41601003: Ward 3,2403.57591,7273.04995,10.931425,0.810545,0.004517,0.008891,0.003986,0.007735,0.000956,...,0.057560,0.010358,0.001421,0.040881,0,0,ZA4161003,-29.142276,25.094093,0.000000
3,41601004: Ward 4,1740.78737,5734.49046,23.119257,0.659914,0.000000,0.006129,0.000000,0.000813,0.037245,...,0.000000,0.000669,0.000000,0.005011,0,0,ZA4161004,-29.372052,24.942867,2.038778
4,41601005: Ward 5,1730.51451,6657.23835,13.652252,0.950575,0.000655,0.001473,0.000598,0.006999,0.000818,...,0.004859,0.001290,0.000673,0.017629,0,0,ZA4161005,-29.409381,25.290165,0.000000


In [3]:
test_df = pd.read_csv('Test.csv')
test_df.head()

,ward,total_households,total_individuals,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,dw_06,...,pw_03,pw_04,pw_05,pw_06,pw_07,pw_08,ADM4_PCODE,lat,lon,NL
0,21001001: Ward 1,2504.95194,8745.15151,0.947257,0.000873,0.002021,0.000000,0.000000,0.030116,0.000452,...,0.001757,0.000000,0.0,0.006649,0.0,0.0,ZA2101001,-32.637758,23.848688,0.000000
1,21001002: Ward 2,2080.27718,7258.11764,0.844993,0.000481,0.043629,0.004714,0.012323,0.012300,0.022132,...,0.000691,0.000000,0.0,0.002916,0.0,0.0,ZA2101002,-31.990536,24.555818,0.000000
2,21001003: Ward 3,1106.62639,5919.13170,0.651380,0.007937,0.007113,0.000000,0.001977,0.259711,0.006505,...,0.002253,0.000000,0.0,0.000000,0.0,0.0,ZA2101003,-32.283595,24.563940,8.269556
3,21001004: Ward 4,2175.56096,10280.57452,0.410837,0.002468,0.011511,0.000485,0.000000,0.449604,0.009256,...,0.000000,0.000661,0.0,0.001379,0.0,0.0,ZA2101004,-32.261612,24.542202,8.626625
4,21001005: Ward 5,1270.83883,6018.34202,0.942851,0.002638,0.000821,0.000000,0.000891,0.000787,0.000830,...,0.000000,0.000000,0.0,0.001660,0.0,0.0,ZA2101005,-32.251571,24.558537,8.601754


In [4]:
Sample = pd.read_csv('SampleSubmission.csv')
Sample.head()

,ward,target
0,21001001: Ward 1,21.87
1,21001002: Ward 2,65.94
2,21001003: Ward 3,83.65
3,21001004: Ward 4,NaN
4,21001005: Ward 5,NaN


In [5]:
train_df.corr()

,total_households,total_individuals,target,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,dw_06,...,pw_02,pw_03,pw_04,pw_05,pw_06,pw_07,pw_08,lat,lon,NL
total_households,1.000000,0.959244,-0.374833,-0.226119,-0.231304,0.222825,0.328743,0.314811,0.095374,0.284317,...,-0.140827,-0.149199,-0.121961,-0.110339,-0.228412,NaN,NaN,0.063018,0.032249,0.708701
total_individuals,0.959244,1.000000,-0.293827,-0.200350,-0.161183,0.178921,0.255194,0.245253,0.109852,0.252280,...,-0.127982,-0.135562,-0.103966,-0.084651,-0.174518,NaN,NaN,0.039653,0.104198,0.684853
target,-0.374833,-0.293827,1.000000,0.018461,0.458206,-0.232727,-0.338137,-0.385533,-0.146357,-0.248573,...,0.442441,0.440941,0.389467,0.349653,0.470676,NaN,NaN,0.120381,0.347088,-0.514398
dw_00,-0.226119,-0.200350,0.018461,1.000000,-0.614702,-0.356335,-0.154956,-0.126421,-0.076181,-0.181262,...,-0.064290,-0.079610,-0.059986,-0.038534,-0.297510,NaN,NaN,0.348089,-0.152692,-0.133895
dw_01,-0.231304,-0.161183,0.458206,-0.614702,1.000000,0.000617,-0.075112,-0.112250,-0.095776,-0.099431,...,0.222130,0.250931,0.202204,0.211378,0.600377,NaN,NaN,-0.346198,0.349469,-0.338058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pw_07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pw_08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lat,0.063018,0.039653,0.120381,0.348089,-0.346198,-0.126910,-0.042252,0.009724,-0.093791,0.015586,...,0.069398,0.114066,0.150117,0.114295,-0.029484,NaN,NaN,1.000000,0.081587,0.049429
lon,0.032249,0.104198,0.347088,-0.152692,0.349469,0.075945,0.028536,-0.028597,-0.135910,-0.052148,...,0.077424,0.075826,0.136322,0.165803,0.325206,NaN,NaN,0.081587,1.000000,-0.026927


In [6]:
length = len(train_df)
new = pd.concat([train_df, test_df])

In [7]:
new = new.drop('ADM4_PCODE', axis=1)

In [8]:
new.head(3)

,NL,car_00,car_01,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,dw_06,...,pw_05,pw_06,pw_07,pw_08,stv_00,stv_01,target,total_households,total_individuals,ward
0,0.292039,0.273727,0.726273,0.933841,0.000846,0.005490,0.000676,0.000000,0.001372,0.005750,...,0.000000,0.012928,0.0,0.0,0.283468,0.716532,16.773757,1674.45058,5888.20750,41601001: Ward 1
1,3.207775,0.144638,0.855362,0.696940,0.001253,0.004402,0.000000,0.002301,0.001323,0.007575,...,0.019092,0.004131,0.0,0.0,0.103641,0.896359,21.496661,1736.99230,6735.33812,41601002: Ward 2
2,0.000000,0.272220,0.727780,0.810545,0.004517,0.008891,0.003986,0.007735,0.000956,0.006686,...,0.001421,0.040881,0.0,0.0,0.165772,0.834228,10.931425,2403.57591,7273.04995,41601003: Ward 3


In [9]:
train_df = new[:length]
test_df = new[length:]
test_df.drop("target",axis=1, inplace=True)

In [10]:
cols  = ['psa_00','car_00','stv_00', 'dw_00', 'dw_01', 'dw_02', 'dw_03', 'dw_04',
       'dw_05', 'dw_06', 'dw_07', 'dw_08', 'dw_09', 'dw_10', 'dw_11', 'dw_12',
       'dw_13', 'lan_00', 'lan_01', 'lan_02', 'lan_03', 'lan_04', 'lan_05',
       'lan_06', 'lan_07', 'lan_08', 'lan_09', 'lan_10', 'lan_11', 'lan_12',
       'lan_13', 'lan_14', 'lat', 'lgt_00', 'lln_00', 'lln_01', 'lon', 'pg_00',
       'pg_01', 'pg_02', 'pg_03','car_01', 'pg_04', 'psa_01', 'psa_02',
       'psa_03', 'psa_04', 'pw_00', 'pw_01', 'pw_02', 'pw_03', 'pw_04',
       'pw_05', 'pw_06', 'pw_07', 'pw_08','NL','stv_01', 'target',
       'total_households', 'total_individuals']

In [11]:
# remove constant features
[feat for feat in train_df[cols] if train_df[feat].std() == 0]


['dw_12', 'dw_13', 'lan_13', 'pw_07', 'pw_08']

In [12]:
train_df.drop(['dw_12', 'dw_13', 'lan_13', 'pw_07', 'pw_08'], axis  =1  ,inplace  = True)
test_df.drop(['dw_12', 'dw_13', 'lan_13', 'pw_07', 'pw_08'], axis  =1  ,inplace  = True)

In [13]:
train_df['ward'].nunique()

2822

In [14]:
train_df['total_households'] = StandardScaler().fit_transform(train_df['total_households'].values.reshape(-1,1))
train_df['total_individuals'] = StandardScaler().fit_transform(train_df['total_individuals'].values.reshape(-1,1))

test_df['total_households'] = StandardScaler().fit_transform(test_df['total_households'].values.reshape(-1,1))
test_df['total_individuals'] = StandardScaler().fit_transform(test_df['total_individuals'].values.reshape(-1,1))

In [15]:
target= train_df["target"]
train =train_df.drop(["ward", "target"], axis=1)
sub_id = Sample["ward"]
test= test_df.drop(["ward"], axis=1)

In [16]:
# train['house_ind'] = train['total_individuals'] / train['total_households']
# test['house_ind'] = test['total_individuals'] / test['total_households']

In [21]:
X = train
y = target

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.7, random_state=1234)

In [23]:
categorical_features_indices = np.where(X.dtypes != np.float)[0]; categorical_features_indices

array([], dtype=int64)

In [24]:
!pip install catboost==0.16.5

In [25]:
from catboost import CatBoostRegressor

In [26]:
# model.fit(X_train, y_train, cat_features=categorical_features_indices, eval_set=(X_val, y_val))

In [32]:
from sklearn.metrics import mean_squared_error

from math import sqrt
from sklearn.model_selection import GridSearchCV
error_test=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
fold=KFold(n_splits=5, random_state=42)
y_pred_error=[]
for train_index, test_index in fold.split(X,y):
    X_train, X_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    
    model=CatBoostRegressor(n_estimators=1000,eval_metric='RMSE',reg_lambda=3,depth=4,learning_rate=0.21202)
    model.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_val, y_val)], early_stopping_rounds=200,verbose=100)
    preds=model.predict(X_val)
    error_test.append(sqrt(mean_squared_error(y_val,preds)))
    pred = model.predict(test)
    y_pred_error.append(pred)

0:	learn: 22.2764592	test: 22.2764592	test1: 17.4931134	best: 17.4931134 (0)	total: 23.6ms	remaining: 23.6s
100:	learn: 3.1394004	test: 3.1394004	test1: 4.0932722	best: 4.0916257 (99)	total: 1.84s	remaining: 16.4s
200:	learn: 2.8048470	test: 2.8048470	test1: 3.8328385	best: 3.8324779 (197)	total: 3.58s	remaining: 14.2s
300:	learn: 2.6173951	test: 2.6173951	test1: 3.7567079	best: 3.7514620 (295)	total: 5.3s	remaining: 12.3s
400:	learn: 2.4849703	test: 2.4849703	test1: 3.7440510	best: 3.7420824 (353)	total: 7.02s	remaining: 10.5s
500:	learn: 2.3554968	test: 2.3554968	test1: 3.7544292	best: 3.7357223 (426)	total: 8.68s	remaining: 8.64s
600:	learn: 2.2622843	test: 2.2622843	test1: 3.7407335	best: 3.7328968 (584)	total: 10.4s	remaining: 6.89s
700:	learn: 2.1443693	test: 2.1443693	test1: 3.7296914	best: 3.7224035 (686)	total: 12.1s	remaining: 5.18s
800:	learn: 2.0709645	test: 2.0709645	test1: 3.7548282	best: 3.7224035 (686)	total: 13.9s	remaining: 3.46s
Stopped by overfitting detector  (200 

In [33]:
Sample.head()

,ward,target
0,21001001: Ward 1,18.211871
1,21001002: Ward 2,11.919515
2,21001003: Ward 3,19.348144
3,21001004: Ward 4,16.402893
4,21001005: Ward 5,21.131954


In [34]:
Sample['ward'] = sub_id
Sample['target'] = np.mean(y_pred_error, 0)

In [35]:
Sample.head()

,ward,target
0,21001001: Ward 1,18.211871
1,21001002: Ward 2,11.919515
2,21001003: Ward 3,19.348144
3,21001004: Ward 4,16.402893
4,21001005: Ward 5,21.131954


In [31]:
Sample.to_csv('lastdays28.csv', index=False)